# Optimize

run and compare optimization frameworks

- hyperopt
- optuna
- nevergrad
- platypus

- scipy? slsqp
pysot
swarmlib

In [1]:
# TODO: add description string to analyze

In [2]:
from datetime import datetime
from functools import partial

import pytest
import numpy as np
import pandas as pd
# import pandas_datareader as pdr

from SWRsimulation import SWRsimulation

In [3]:
N_TRIALS = 1000

In [4]:
def datestr():
    return datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S")


In [5]:
# load Damodaran data from pickle (via http://people.stern.nyu.edu/adamodar/New_Home_Page/datafile/histretSP.html )

RETURN_FILE = 'histretSP'
def load_returns():
    return pd.read_pickle('%s.pickle' % RETURN_FILE)

download_df = load_returns()
return_df = download_df.iloc[:, [0, 3, 12]]
return_df.columns=['stocks', 'bonds', 'cpi']

return_df

,stocks,bonds,cpi
Year,,,
1928,0.438112,0.032196,-0.011522
1929,-0.082979,0.030179,0.000000
1930,-0.251236,0.005398,-0.026712
1931,-0.438375,-0.156808,-0.089321
1932,-0.086424,0.235896,-0.103014
...,...,...,...
2016,0.117731,0.103651,0.012616
2017,0.216055,0.097239,0.021301
2018,-0.042269,-0.027626,0.024426


In [6]:
# should adjust CPI to year-ending also but leave it for now
real_return_df = return_df.copy()
# real_return_df.loc[1948:, 'cpi'] = cpi_test['cpi_fred']
# adjust returns for inflation
real_return_df['stocks'] = (1 + real_return_df['stocks']) / (1 + real_return_df['cpi']) - 1
real_return_df['bonds'] = (1 + real_return_df['bonds']) / (1 + real_return_df['cpi']) - 1
real_return_df.drop('cpi', axis=1, inplace=True)
real_return_df.to_pickle('real_return_df.pickle')

real_return_df

,stocks,bonds
Year,,
1928,0.454874,0.044227
1929,-0.082979,0.030179
1930,-0.230686,0.032991
1931,-0.383290,-0.074106
1932,0.018495,0.377832
...,...,...
2016,0.103805,0.089901
2017,0.190692,0.074354
2018,-0.065104,-0.050811


In [7]:
FIXED_SPEND = 2
VARIABLE_SPEND = 2
ALLOC_STOCKS = 0.6
N_RET_YEARS = 30
import pdb
from functools import partial

def ce_experiment(n_ret_years, gamma, *params):
    # some optimizers pass an array, some pass a dict, some pass indiv unpacked params
    if len(params) == 1:   # unpack
        params = params[0]
        
    if type(params) == dict:
        fixed_spend = params['fixed_spend']
        variable_spend = params['variable_spend']
        alloc_stocks = params['alloc_stocks']
        n_ret_years = N_RET_YEARS
    elif type(params) == list or type(params) == tuple:
        fixed_spend, variable_spend, alloc_stocks = params     
        n_ret_years = N_RET_YEARS
    elif type(params) == np.ndarray:
        fixed_spend, variable_spend, alloc_stocks = params.tolist()
        n_ret_years = N_RET_YEARS
    else:
        raise Exception('bad argument to ce_experiment', type(params), params)
        
    alloc_bonds = 1.0 - alloc_stocks

    s = SWRsimulation.SWRsimulation({
        'simulation': {'returns_df': real_return_df,
                       'n_ret_years': n_ret_years,
                       'montecarlo': N_TRIALS,
                       'montecarlo_replacement': False,
                      },
        'allocation': {'asset_weights': np.array([alloc_stocks, alloc_bonds])}, 
        'withdrawal': {'fixed_pct': fixed_spend,
                       'variable_pct': variable_spend},
        'evaluation': {},
        'analysis': {}    
    })
    s.simulate()
    return -SWRsimulation.crra_ce(np.array([z['ce_spend'] for z in s.latest_simulation]), gamma)

# freeze first 2 args with partial
ce_experiment_1 = partial(ce_experiment, N_RET_YEARS, 1)

# unpacked args
print(ce_experiment_1(FIXED_SPEND, VARIABLE_SPEND, ALLOC_STOCKS))

# list arg
print(ce_experiment_1([FIXED_SPEND, VARIABLE_SPEND, ALLOC_STOCKS]))

# ndarray arg
print(ce_experiment_1(np.array([FIXED_SPEND, VARIABLE_SPEND, ALLOC_STOCKS])))

# dict arg
print(ce_experiment_1({'fixed_spend': FIXED_SPEND, 
                       'variable_spend': VARIABLE_SPEND, 
                       'alloc_stocks': ALLOC_STOCKS}))



-152491.48291901182
-154292.8257121518
-153301.913616406
-150272.62026148135


In [8]:
import optuna   # https://optuna.org/
optuna.logging.set_verbosity(optuna.logging.ERROR)

def objective(gamma, trial):
    fixed_spend = trial.suggest_uniform('fixed_spend', 0, 5)
    variable_spend = trial.suggest_uniform('variable_spend', 0, 10)
    alloc_stocks = trial.suggest_uniform('alloc_stocks', 0, 1)
    oobjective = partial(ce_experiment, N_RET_YEARS, gamma)
    return oobjective([fixed_spend, variable_spend, alloc_stocks])

study_params = []
study_values = []
study_gammas = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16]

n_trials = N_TRIALS
optimizer = "optuna"

for gamma in study_gammas:
    print("%s Starting %s, gamma %d, %d trials" % (datestr(), optimizer, gamma, n_trials))
    start_time = datetime.now() 
    study = optuna.create_study()
    study.optimize(partial(objective, gamma), n_trials=n_trials)
    study_params.append(study.best_params)
    study_values.append(study.best_value)
    time_elapsed = datetime.now() - start_time 
    
    print("%s Finishing optimizer %s, gamma %d, %d trials in %s" % (datestr(), optimizer, gamma, n_trials, time_elapsed))
    print("%s best value %f, best params %s" % (datestr(), study.best_value, study.best_params))
    

2021-02-07 16:38:35 Starting optuna, gamma 0, 1000 trials
2021-02-07 17:11:25 Finishing optimizer optuna, gamma 0, 1000 trials in 0:32:50.068542
2021-02-07 17:11:25 best value -266183.902826, best params {'fixed_spend': 0.33557093657243237, 'variable_spend': 5.918144621693734, 'alloc_stocks': 0.9993270701039008}
2021-02-07 17:11:25 Starting optuna, gamma 1, 1000 trials
2021-02-07 17:43:58 Finishing optimizer optuna, gamma 1, 1000 trials in 0:32:33.099398
2021-02-07 17:43:58 best value -222222.038342, best params {'fixed_spend': 0.7781470922952266, 'variable_spend': 5.921319587544653, 'alloc_stocks': 0.9820930186717508}
2021-02-07 17:43:58 Starting optuna, gamma 2, 1000 trials
2021-02-07 18:16:55 Finishing optimizer optuna, gamma 2, 1000 trials in 0:32:56.784303
2021-02-07 18:16:55 best value -196566.220203, best params {'fixed_spend': 0.5300850734566511, 'variable_spend': 6.874827240420751, 'alloc_stocks': 0.9787957370966124}
2021-02-07 18:16:55 Starting optuna, gamma 4, 1000 trials
20

KeyboardInterrupt: 

In [ ]:
results = pd.DataFrame(study_params)
results['gamma'] = study_gammas
results['value'] = study_values
results['alloc_bonds'] = 1-results['alloc_stocks']
results=results[['gamma', 'alloc_stocks', 'alloc_bonds', 'fixed_spend', 'variable_spend','value']]
results

In [ ]:
resultsrow = results.iloc[1]
FIXED_SPEND = resultsrow.fixed_spend
VARIABLE_SPEND = resultsrow.variable_spend
ALLOC_STOCKS = resultsrow.alloc_stocks
GAMMA = resultsrow.gamma
N_RET_YEARS = 30

ce_experiment(N_RET_YEARS,
              GAMMA,
              {'fixed_spend': FIXED_SPEND, 
               'variable_spend': VARIABLE_SPEND, 
               'alloc_stocks': ALLOC_STOCKS})


In [ ]:
resultsrow = results.iloc[-1]
FIXED_SPEND = resultsrow.fixed_spend
VARIABLE_SPEND = resultsrow.variable_spend
ALLOC_STOCKS = resultsrow.alloc_stocks
ALLOC_BONDS = 1 - ALLOC_STOCKS
GAMMA = resultsrow.gamma
N_RET_YEARS = 30

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': N_RET_YEARS,
#                    'montecarlo': NTRIALS,
#                    'montecarlo_replacement': True,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, # default is equal-weight 
    'withdrawal': {'fixed_pct': FIXED_SPEND,
                   'variable_pct': VARIABLE_SPEND},
    'evaluation': {}, # no args, default = number of years to exhaustion
    'analysis': {'histogram': True}    
})

print(s)

s.simulate()

s.analyze()

In [ ]:
import hyperopt  # http://hyperopt.github.io/hyperopt/
from hyperopt import fmin, tpe, hp

n_trials = N_TRIALS
optimizer = "hyperopt"

study_params = []
study_gammas = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16]
for gamma in study_gammas:
    print("%s Starting %s, gamma %d, %d trials" % (datestr(), optimizer, gamma, n_trials))
    start_time = datetime.now() 
    hobjective = partial(ce_experiment, N_RET_YEARS, gamma)
    best = fmin(fn=hobjective,
                space={'fixed_spend': hp.uniform('fixed_spend', 0, 5),
                       'variable_spend': hp.uniform('variable_spend', 0, 10),
                       'alloc_stocks': hp.uniform('alloc_stocks', 0, 1),
                      },
                algo=tpe.suggest,
                max_evals=n_trials)
    study_params.append(best)
    time_elapsed = datetime.now() - start_time 
    print("%s Finishing optimizer %s, gamma %d, %d trials in %s" % (datestr(), optimizer, gamma, n_trials, time_elapsed))
    print("%s best params %s" % (datestr(), study.best_params))
        

In [ ]:
results = pd.DataFrame(study_params)
results['gamma'] = study_gammas
results['alloc_bonds'] = 1-results['alloc_stocks']
results=results[['gamma', 'alloc_stocks', 'alloc_bonds', 'fixed_spend', 'variable_spend']]
study_values = []
for t in results.itertuples():
    hobjective = partial(ce_experiment, N_RET_YEARS, t.gamma)
    study_values.append(hobjective({'fixed_spend': t.fixed_spend, 
                                    'variable_spend': t.variable_spend, 
                                    'alloc_stocks': t.alloc_stocks, 
                                   }))
results['value'] = study_values

results

In [ ]:
resultsrow = results.iloc[1]
FIXED_SPEND = resultsrow.fixed_spend
VARIABLE_SPEND = resultsrow.variable_spend
ALLOC_STOCKS = resultsrow.alloc_stocks
GAMMA = resultsrow.gamma
N_RET_YEARS = 30

ce_experiment(N_RET_YEARS,
              GAMMA,
              {'fixed_spend': FIXED_SPEND, 
               'variable_spend': VARIABLE_SPEND, 
               'alloc_stocks': ALLOC_STOCKS})


In [ ]:
resultsrow = results.iloc[-1]
FIXED_SPEND = resultsrow.fixed_spend
VARIABLE_SPEND = resultsrow.variable_spend
ALLOC_STOCKS = resultsrow.alloc_stocks
ALLOC_BONDS = 1 - ALLOC_STOCKS
GAMMA = resultsrow.gamma
N_RET_YEARS = 30

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': N_RET_YEARS,
#                    'montecarlo': NTRIALS,
#                    'montecarlo_replacement': True,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, # default is equal-weight 
    'withdrawal': {'fixed_pct': FIXED_SPEND,
                   'variable_pct': VARIABLE_SPEND},
    'evaluation': {}, # no args, default = number of years to exhaustion
    'analysis': {'histogram': True}    
})

print(s)

s.simulate()
s.analyze()

In [ ]:
# https://github.com/Project-Platypus/Platypus
from platypus import MOEAD, Problem, Real

n_trials = N_TRIALS
optimizer = "platypus (MOEAD)"

study_params = []
study_gammas = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16]

for gamma in study_gammas:
    print("%s Starting %s, gamma %d, %d trials" % (datestr(), optimizer, gamma, n_trials))
    start_time = datetime.now() 
    # number inputs, outputs
    problem = Problem(3, 1)
    # ranges
    problem.types[0] = Real(0, 5)   # fixed_spend
    problem.types[1] = Real(0, 10)  # variable_spend
    problem.types[2] = Real(0, 1)   # alloc_stocks

    # platypus wants a numpy array return because it's a multi-objective algo
    f = partial(ce_experiment, N_RET_YEARS, gamma)
    def pobjective(params):
        return np.array([f(params)])

    problem.function = pobjective
    algorithm = MOEAD(problem)
    algorithm.run(n_trials)
    best = sorted([(s.objectives[0], s.variables) for s in algorithm.result if s.feasible])[0]
    # tuple of best objective value, best params
    study_params.append(best)
    time_elapsed = datetime.now() - start_time 
    print("%s Finishing optimizer %s, gamma %d, %d trials in %s" % (datestr(), optimizer, gamma, n_trials, time_elapsed))
    print("%s best params %s" % (datestr(), best))


In [ ]:
results = pd.DataFrame([s[1] for s in study_params], columns=['fixed_spend', 'variable_spend', 'alloc_stocks'])
results['gamma'] = study_gammas
results['alloc_bonds'] = 1-results['alloc_stocks']
results=results[['gamma', 'alloc_stocks', 'alloc_bonds', 'fixed_spend', 'variable_spend']]
results['value'] = [s[0] for s in study_params]
results

In [ ]:
resultsrow = results.iloc[1]
FIXED_SPEND = resultsrow.fixed_spend
VARIABLE_SPEND = resultsrow.variable_spend
ALLOC_STOCKS = resultsrow.alloc_stocks
GAMMA = resultsrow.gamma
N_RET_YEARS = 30

ce_experiment(N_RET_YEARS,
              GAMMA,
              {'fixed_spend': FIXED_SPEND, 
               'variable_spend': VARIABLE_SPEND, 
               'alloc_stocks': ALLOC_STOCKS})


In [ ]:
resultsrow = results.iloc[-1]
FIXED_SPEND = resultsrow.fixed_spend
VARIABLE_SPEND = resultsrow.variable_spend
ALLOC_STOCKS = resultsrow.alloc_stocks
ALLOC_BONDS = 1 - ALLOC_STOCKS
GAMMA = resultsrow.gamma
N_RET_YEARS = 30

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': N_RET_YEARS,
#                    'montecarlo': NTRIALS,
#                    'montecarlo_replacement': True,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, # default is equal-weight 
    'withdrawal': {'fixed_pct': FIXED_SPEND,
                   'variable_pct': VARIABLE_SPEND},
    'evaluation': {}, # no args, default = number of years to exhaustion
    'analysis': {'histogram': True}    
})

print(s)

s.simulate()
s.analyze()

In [ ]:
import nevergrad as ng
from concurrent import futures

# https://facebookresearch.github.io/nevergrad/optimization.html

n_trials = N_TRIALS
optimizer = "nevergrad (OnePlusOne)"

study_params = []
study_gammas = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16]

for gamma in study_gammas:

    print("%s Starting %s, gamma %d, %d trials" % (datestr(), optimizer, gamma, n_trials))
    start_time = datetime.now() 
    nobjective = partial(ce_experiment, N_RET_YEARS, gamma)
    optimizer = ng.optimizers.OnePlusOne(parametrization=3, budget=n_trials, num_workers=10)

    optimizer.parametrization.register_cheap_constraint(lambda x: x[0] >= 0)
    optimizer.parametrization.register_cheap_constraint(lambda x: x[0] <= 5)
    optimizer.parametrization.register_cheap_constraint(lambda x: x[1] >= 0)
    optimizer.parametrization.register_cheap_constraint(lambda x: x[1] <= 10)
    optimizer.parametrization.register_cheap_constraint(lambda x: x[2] >= 0)
    optimizer.parametrization.register_cheap_constraint(lambda x: x[2] <= 1)
    with futures.ThreadPoolExecutor(max_workers=optimizer.num_workers) as executor:
        recommendation = optimizer.minimize(nobjective, executor=executor, batch_mode=False)
    
    time_elapsed = datetime.now() - start_time 
    study_params.append((recommendation.loss, recommendation.value.tolist()))

    print("%s Finishing optimizer %s, gamma %d, %d trials in %s" % (datestr(), optimizer, gamma, n_trials, time_elapsed))
    print("%s best params %s value %f" % (datestr(), recommendation.value, recommendation.loss))
        


In [ ]:
results = pd.DataFrame([s[1] for s in study_params], columns=['fixed_spend', 'variable_spend', 'alloc_stocks'])
results['gamma'] = study_gammas
results['alloc_bonds'] = 1-results['alloc_stocks']
results=results[['gamma', 'alloc_stocks', 'alloc_bonds', 'fixed_spend', 'variable_spend']]
results['value'] = [s[0] for s in study_params]
results


In [ ]:
resultsrow = results.iloc[1]
FIXED_SPEND = resultsrow.fixed_spend
VARIABLE_SPEND = resultsrow.variable_spend
ALLOC_STOCKS = resultsrow.alloc_stocks
GAMMA = resultsrow.gamma
N_RET_YEARS = 30

ce_experiment(N_RET_YEARS,
              GAMMA,
              {'fixed_spend': FIXED_SPEND, 
               'variable_spend': VARIABLE_SPEND, 
               'alloc_stocks': ALLOC_STOCKS})


In [ ]:
resultsrow = results.iloc[-1]
FIXED_SPEND = resultsrow.fixed_spend
VARIABLE_SPEND = resultsrow.variable_spend
ALLOC_STOCKS = resultsrow.alloc_stocks
ALLOC_BONDS = 1 - ALLOC_STOCKS
GAMMA = resultsrow.gamma
N_RET_YEARS = 30

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': N_RET_YEARS,
#                    'montecarlo': NTRIALS,
#                    'montecarlo_replacement': True,
                  },
    'allocation': {'asset_weights': np.array([0.5, 0.5])}, # default is equal-weight 
    'withdrawal': {'fixed_pct': FIXED_SPEND,
                   'variable_pct': VARIABLE_SPEND},
    'evaluation': {}, # no args, default = number of years to exhaustion
    'analysis': {'histogram': True}    
})

print(s)

s.simulate()
s.analyze()

In [ ]:
# TODO: 
# chart optimizer values vs. gamma, show they are similar
# chart times and values , highlight besta
# try various nevergrad, platypus options

# analyze should compute ce spend over history
# write a function to take 4 parameters, fixed variable stocks bonds and return CE over history
# run optimizer on function



In [ ]:
import math
gamma=4
crra_utility = lambda x: (x ** (1 - gamma) - 1) / (1 - gamma)
print(crra_utility (1000000000))
winbet = 110
losebet = 100
w = 1000

win_utils = crra_utility(w + winbet) - crra_utility(w)
loss_utils = crra_utility(w) - crra_utility(w - losebet)
print ('win', win_utils)
print ('loss', loss_utils)
print('edge', win_utils-loss_utils)


In [ ]:
winbet = 2000
losebet = 1000
w = 4000

win_utils = crra_utility(w + winbet) - crra_utility(w)
loss_utils = crra_utility(w) - crra_utility(w - losebet)
print ('win', win_utils)
print ('loss', loss_utils)
print('edge', win_utils-loss_utils)


In [ ]:
def general_ce(cashflows, gamma):
    cashflows = np.longdouble(cashflows)
    if gamma == 1:
        u = np.mean(np.log(cashflows))
        ce = np.exp(u)
    else:
        u = np.mean((cashflows ** (1 - gamma) - 1) / (1 - gamma))
        ce = (1 + u * (1 - gamma)) ** (1 / (1 - gamma))
    ce = np.float(ce)
    return ce * len(cashflows)

In [ ]:
z = pd.read_csv('log.csv')

In [ ]:
z.columns

In [ ]:
with pd.option_context('display.max_rows', None):
    display(z.sort_values(['optimizer', 'objective', 'n_dim']))

In [ ]:
z.columns

In [ ]:
zz = z[['objective', 'n_dim', 'optimizer', 'best_value']].iloc[z[['objective', 'n_dim', 'best_value']].groupby(['objective', 'n_dim']).idxmax().values[:,0]]
zz.groupby('optimizer').count()
